Step 1 Import Liblary

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
from warnings import simplefilter
import plotly.graph_objects as go
import time

simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)


Step 2, Download Data dari Pustaka Yahoo Finance

In [ ]:
stock_symbol = "GC=F"
start_date = "2022-08-01"
end_date = "2024-08-01"
price_type = "Close"

data = yf.download(stock_symbol, start=start_date, end=end_date)

# Scaling Data
close_prices = data[price_type].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices)


[*********************100%%**********************]  1 of 1 completed


Step 3, Preprocessing Data

In [ ]:
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:(i + n_steps), 0])
        y.append(data[i + n_steps, 0])
    return np.array(X), np.array(y)

n_steps = 30
X, y = prepare_data(scaled_data, n_steps)

#Spliting Data 80/20
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


Step 4 Membangun dan Melatih Model

In [ ]:
def build_model(params, X_train):
    model = Sequential()
    model.add(LSTM(units=params['units'], return_sequences=True, activation='tanh', input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=params['units'], activation='tanh'))
    model.add(Dense(units=1))

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model


In [ ]:
# Definisikan parameter untuk epoch dan batch size
epoch_options = [50, 100, 150, 200]
batch_size_options = [16, 32, 64, 128]

best_rmse = float('inf')
best_params = None
results = []

# Loop untuk setiap kombinasi epoch dan batch size
for e in epoch_options:
    for b in batch_size_options:
        print(f"Training with Epochs: {e}, Batch Size: {b}")
        params = {'units': 100, 'epochs': e, 'batch_size': b}
        model = build_model(params, X_train)

        # Melatih model
        history = model.fit(X_train, y_train,
                            epochs=e,
                            batch_size=b,
                            verbose=2,
                            validation_split=0.1)

        # Melakukan prediksi
        y_pred = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred)
        y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Menghitung MSE, RMSE, MAPE, dan MAE
        mse = mean_squared_error(y_test_orig, y_pred)
        rmse = math.sqrt(mse)
        mae = mean_absolute_error(y_test_orig, y_pred)
        mape = np.mean(np.abs((y_test_orig - y_pred) / y_test_orig)) * 100

        # Simpan hasil evaluasi untuk setiap kombinasi parameter
        results.append((e, b, mse, rmse, mae, mape))

        # Update model terbaik jika RMSE saat ini lebih baik
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = params
            best_model = model
            best_history = history
            best_y_pred = y_pred
            best_y_test_orig = y_test_orig

# Cetak hasil evaluasi
print(f"Best RMSE: {best_rmse} with params: {best_params}")
print("\nDetailed results for each parameter combination:")

# Menampilkan hasil untuk setiap kombinasi parameter yang dilatih
for result in results:
    e, b, mse, rmse, mae, mape = result
    print(f"Epochs: {e}, Batch Size: {b} => MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}%")

Training with Epochs: 50, Batch Size: 16


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
22/22 - 5s - 231ms/step - loss: 0.0172 - val_loss: 0.0034
Epoch 2/50
22/22 - 1s - 64ms/step - loss: 0.0033 - val_loss: 0.0036
Epoch 3/50
22/22 - 1s - 57ms/step - loss: 0.0024 - val_loss: 0.0046
Epoch 4/50
22/22 - 1s - 32ms/step - loss: 0.0021 - val_loss: 0.0035
Epoch 5/50
22/22 - 1s - 57ms/step - loss: 0.0019 - val_loss: 0.0028
Epoch 6/50
22/22 - 1s - 58ms/step - loss: 0.0020 - val_loss: 0.0039
Epoch 7/50
22/22 - 1s - 57ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 8/50
22/22 - 1s - 33ms/step - loss: 0.0018 - val_loss: 0.0045
Epoch 9/50
22/22 - 1s - 58ms/step - loss: 0.0016 - val_loss: 0.0034
Epoch 10/50
22/22 - 1s - 61ms/step - loss: 0.0016 - val_loss: 0.0026
Epoch 11/50
22/22 - 1s - 55ms/step - loss: 0.0015 - val_loss: 0.0021
Epoch 12/50
22/22 - 1s - 56ms/step - loss: 0.0014 - val_loss: 0.0023
Epoch 13/50
22/22 - 1s - 60ms/step - loss: 0.0014 - val_loss: 0.0016
Epoch 14/50
22/22 - 1s - 49ms/step - loss: 0.0013 - val_loss: 0.0017
Epoch 15/50
22/22 - 1s - 46ms/step - loss:

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 - 4s - 342ms/step - loss: 0.0277 - val_loss: 0.0275
Epoch 2/50
11/11 - 1s - 46ms/step - loss: 0.0056 - val_loss: 0.0031
Epoch 3/50
11/11 - 0s - 44ms/step - loss: 0.0035 - val_loss: 0.0051
Epoch 4/50
11/11 - 1s - 56ms/step - loss: 0.0025 - val_loss: 0.0051
Epoch 5/50
11/11 - 1s - 46ms/step - loss: 0.0021 - val_loss: 0.0035
Epoch 6/50
11/11 - 1s - 55ms/step - loss: 0.0020 - val_loss: 0.0028
Epoch 7/50
11/11 - 1s - 46ms/step - loss: 0.0019 - val_loss: 0.0035
Epoch 8/50
11/11 - 0s - 45ms/step - loss: 0.0019 - val_loss: 0.0036
Epoch 9/50
11/11 - 1s - 58ms/step - loss: 0.0018 - val_loss: 0.0040
Epoch 10/50
11/11 - 0s - 44ms/step - loss: 0.0018 - val_loss: 0.0027
Epoch 11/50
11/11 - 1s - 47ms/step - loss: 0.0018 - val_loss: 0.0037
Epoch 12/50
11/11 - 1s - 63ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 13/50
11/11 - 1s - 76ms/step - loss: 0.0018 - val_loss: 0.0020
Epoch 14/50
11/11 - 1s - 72ms/step - loss: 0.0017 - val_loss: 0.0028
Epoch 15/50
11/11 - 1s - 111ms/step - loss: 0.0015 - 

...

11/11 - 5s - 445ms/step - loss: 0.0179 - val_loss: 0.0242
Epoch 2/200
11/11 - 2s - 149ms/step - loss: 0.0046 - val_loss: 0.0070
Epoch 3/200
11/11 - 1s - 55ms/step - loss: 0.0031 - val_loss: 0.0028
Epoch 4/200
11/11 - 1s - 47ms/step - loss: 0.0024 - val_loss: 0.0050
Epoch 5/200
11/11 - 1s - 45ms/step - loss: 0.0020 - val_loss: 0.0046
Epoch 6/200
11/11 - 1s - 56ms/step - loss: 0.0020 - val_loss: 0.0028
Epoch 7/200
11/11 - 1s - 46ms/step - loss: 0.0018 - val_loss: 0.0030
Epoch 8/200
11/11 - 1s - 46ms/step - loss: 0.0018 - val_loss: 0.0045
Epoch 9/200
11/11 - 1s - 47ms/step - loss: 0.0018 - val_loss: 0.0034
Epoch 10/200
11/11 - 1s - 55ms/step - loss: 0.0017 - val_loss: 0.0024
Epoch 11/200
11/11 - 1s - 47ms/step - loss: 0.0016 - val_loss: 0.0024
Epoch 12/200
11/11 - 1s - 54ms/step - loss: 0.0016 - val_loss: 0.0029
Epoch 13/200
11/11 - 1s - 48ms/step - loss: 0.0016 - val_loss: 0.0030
Epoch 14/200
11/11 - 1s - 46ms/step - loss: 0.0015 - val_loss: 0.0037
Epoch 15/200
11/11 - 1s - 58ms/step - l

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 - 5s - 866ms/step - loss: 0.0549 - val_loss: 0.0186
Epoch 2/200
6/6 - 1s - 90ms/step - loss: 0.0088 - val_loss: 0.0292
Epoch 3/200
6/6 - 1s - 102ms/step - loss: 0.0081 - val_loss: 0.0082
Epoch 4/200
6/6 - 1s - 105ms/step - loss: 0.0038 - val_loss: 0.0030
Epoch 5/200
6/6 - 1s - 85ms/step - loss: 0.0032 - val_loss: 0.0096
Epoch 6/200
6/6 - 1s - 106ms/step - loss: 0.0031 - val_loss: 0.0059
Epoch 7/200
6/6 - 1s - 89ms/step - loss: 0.0025 - val_loss: 0.0031
Epoch 8/200
6/6 - 1s - 107ms/step - loss: 0.0023 - val_loss: 0.0053
Epoch 9/200
6/6 - 1s - 88ms/step - loss: 0.0022 - val_loss: 0.0034
Epoch 10/200
6/6 - 1s - 102ms/step - loss: 0.0021 - val_loss: 0.0034
Epoch 11/200
6/6 - 1s - 105ms/step - loss: 0.0020 - val_loss: 0.0038
Epoch 12/200
6/6 - 1s - 161ms/step - loss: 0.0020 - val_loss: 0.0031
Epoch 13/200
6/6 - 1s - 166ms/step - loss: 0.0020 - val_loss: 0.0031
Epoch 14/200
6/6 - 1s - 154ms/step - loss: 0.0019 - val_loss: 0.0032
Epoch 15/200
6/6 - 1s - 176ms/step - loss: 0.0019 - val_los

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 - 4s - 1s/step - loss: 0.0959 - val_loss: 0.0652
Epoch 2/200
3/3 - 0s - 132ms/step - loss: 0.0126 - val_loss: 0.0218
Epoch 3/200
3/3 - 1s - 210ms/step - loss: 0.0167 - val_loss: 0.0048
Epoch 4/200
3/3 - 1s - 208ms/step - loss: 0.0042 - val_loss: 0.0272
Epoch 5/200
3/3 - 1s - 218ms/step - loss: 0.0089 - val_loss: 0.0256
Epoch 6/200
3/3 - 1s - 282ms/step - loss: 0.0064 - val_loss: 0.0101
Epoch 7/200
3/3 - 1s - 421ms/step - loss: 0.0032 - val_loss: 0.0029
Epoch 8/200
3/3 - 1s - 248ms/step - loss: 0.0047 - val_loss: 0.0028
Epoch 9/200
3/3 - 1s - 218ms/step - loss: 0.0039 - val_loss: 0.0051
Epoch 10/200
3/3 - 0s - 139ms/step - loss: 0.0028 - val_loss: 0.0101
Epoch 11/200
3/3 - 0s - 140ms/step - loss: 0.0032 - val_loss: 0.0101
Epoch 12/200
3/3 - 0s - 141ms/step - loss: 0.0029 - val_loss: 0.0061
Epoch 13/200
3/3 - 0s - 136ms/step - loss: 0.0024 - val_loss: 0.0035
Epoch 14/200
3/3 - 1s - 211ms/step - loss: 0.0025 - val_loss: 0.0030
Epoch 15/200
3/3 - 0s - 136ms/step - loss: 0.0023 - val_lo

Step 5 Visualisasi Hasil Prediksi

In [ ]:
def visualize_predictions(data, train_size, n_steps, y_test_orig, y_pred):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data.index[:train_size + n_steps],
                             y=data['Close'].values[:train_size + n_steps],
                             mode='lines',
                             name="Training Data",
                             line=dict(color='gray')))

    fig.add_trace(go.Scatter(x=data.index[train_size + n_steps:],
                             y=y_test_orig.flatten(),
                             mode='lines',
                             name="Actual Stock Prices",
                             line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=data.index[train_size + n_steps:],
                             y=y_pred.flatten(),
                             mode='lines',
                             name="Predicted Stock Prices",
                             line=dict(color='red')))

    fig.update_layout(title="Gold Currency Price Prediction",
                      xaxis_title="Date",
                      yaxis_title="Price",
                      template='plotly_dark')

    fig.show()

visualize_predictions(data, train_size, n_steps, best_y_test_orig, best_y_pred)


Step 6 Prediksi Masa Depan dengan Model Terbaik

In [ ]:
def forecast_future_prices(model, scaler, scaled_data, n_steps, forecast_days):
    last_sequence = scaled_data[-n_steps:]
    future_predictions = []

    for _ in range(forecast_days):
        last_sequence = last_sequence.reshape((1, n_steps, 1))
        predicted_price = model.predict(last_sequence)
        future_predictions.append(predicted_price[0, 0])
        last_sequence = np.append(last_sequence[:, 1:, :], predicted_price.reshape((1, 1, 1)), axis=1)

    future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))
    return future_predictions.flatten()

forecast_days = 30 #Ganti Sesuai Kebutuhan
future_predictions = forecast_future_prices(best_model, scaler, scaled_data, n_steps, forecast_days)

def visualize_future_predictions(data, future_predictions, forecast_days):
    last_date = data.index[-1]
    last_close_price = data['Close'].values[-1]
    future_predictions = np.insert(future_predictions, 0, last_close_price)
    future_dates = pd.date_range(last_date, periods=forecast_days + 1)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data.index,
                             y=data['Close'].values,
                             mode='lines',
                             name="Actual Stock Prices",
                             line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=future_dates,
                             y=future_predictions,
                             mode='lines',
                             name="Future Predictions",
                             line=dict(color='green')))

    fig.update_layout(title="Prediksi Masa Depan",
                      xaxis_title="Date",
                      yaxis_title="Price",
                      template='plotly_dark')

    fig.show()

visualize_future_predictions(data, future_predictions, forecast_days)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━